Getting started with Spacy<br>
Import data.<br>
We repeat the preprocessing from the previous homework.

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import os

In [3]:
os.chdir( "/content/gdrive/MyDrive/flair" ) 

In [4]:
path_to_data = os.getcwd() + '/Entity Recognition in Resumes.json'

In [5]:
myfile = open( path_to_data, "r", encoding = "utf-8" )

imported_data = []

for datum in myfile:
    
  # process data as list
  imported_data.append(datum)

myfile.close()

# print first line
print("First Line:\t", imported_data[0])

# print how many resumees were read in
print("Total Number of Resumes:\t", len(imported_data))


First Line:	 {"content": "Afreen Jamadar\nActive member of IIIT Committee in Third year\n\nSangli, Maharashtra - Email me on Indeed: indeed.com/r/Afreen-Jamadar/8baf379b705e37c6\n\nI wish to use my knowledge, skills and conceptual understanding to create excellent team\nenvironments and work consistently achieving organization objectives believes in taking initiative\nand work to excellence in my work.\n\nWORK EXPERIENCE\n\nActive member of IIIT Committee in Third year\n\nCisco Networking -  Kanpur, Uttar Pradesh\n\norganized by Techkriti IIT Kanpur and Azure Skynet.\nPERSONALLITY TRAITS:\n• Quick learning ability\n• hard working\n\nEDUCATION\n\nPG-DAC\n\nCDAC ACTS\n\n2017\n\nBachelor of Engg in Information Technology\n\nShivaji University Kolhapur -  Kolhapur, Maharashtra\n\n2016\n\nSKILLS\n\nDatabase (Less than 1 year), HTML (Less than 1 year), Linux. (Less than 1 year), MICROSOFT\nACCESS (Less than 1 year), MICROSOFT WINDOWS (Less than 1 year)\n\nADDITIONAL INFORMATION\n\nTECHNICAL 

In [6]:
import json

In [7]:
mapped_data = [ json.loads( datum ) for datum in imported_data  ]

In [8]:
## data conversion method
def convert_data(data):
    """
    Creates NER training data in Spacy format from JSON dataset
    Outputs the Spacy training data which can be used for Spacy training.
    """
    text = data['content']
    entities = []
    if data['annotation'] is not None:
        for annotation in data['annotation']:
            # only a single point in text annotation.
            point = annotation['points'][0]
            labels = annotation['label']
            # handle both list of labels or a single label.
            if not isinstance(labels, list):
                labels = [labels]
            for label in labels:
                # dataturks indices are both inclusive [start, end] but spacy is not [start, end)
                entities.append((point['start'], point['end'] + 1, label))
    return (text, {"entities": entities})
   
## Using a loop or list comprehension, convert each resume in mapped_data using the convert function above, 
## storing the result
converted_resumes = [convert_data(resume) for resume in mapped_data]
## print the number of resumes in converted resumes 
print("Converted resumes:\t", len(converted_resumes))

Converted resumes:	 701


In [9]:
# filter out the resumees whose entities have no entries.
converted_complete_resumees = [resume for resume in converted_resumes if len(resume[1]["entities"]) > 0]

Up until now, you could reuse the code from the previous notebook, now, something new comes.

In [10]:
import spacy
nlp = spacy.load('en')
print(nlp)

__nlp__ is Spacy's English language model. For this model, a pretrained NER-model exists.

In [11]:
ner = nlp.get_pipe('ner')
labels = ner.labels
print(labels)

('CARDINAL', 'DATE', 'EVENT', 'FAC', 'GPE', 'LANGUAGE', 'LAW', 'LOC', 'MONEY', 'NORP', 'ORDINAL', 'ORG', 'PERCENT', 'PERSON', 'PRODUCT', 'QUANTITY', 'TIME', 'WORK_OF_ART')


__Question 1__: Explain the labels __GPE__, __FAC__, __NORP__.<br>

The labbels refer to Countries, cities, states (GPE) , buildings( FAC) and Nationalities or Social Groups (NORP)

Which of these labels from __ner__ do you think will Spacy recognize in the resumees?<br>
__Task 1__: choose a resumee.

In [12]:

for label in ['GPE', 'FAC', 'NORP']:
    print(label, ":\t", spacy.explain(label))

GPE :	 Countries, cities, states
FAC :	 Buildings, airports, highways, bridges, etc.
NORP :	 Nationalities or religious or political groups


In [13]:
# get a single resume text and print it out.
resume = 24
restxt = converted_complete_resumees[resume][0]
## print it out, removing extraneous spaces
print("\n".join(restxt.split('\n\n')))

Karthihayini C
Systems Engineer - Infosys Limited
Rajapalaiyam, Tamil Nadu - Email me on Indeed: indeed.com/r/Karthihayini-
C/627254c443836b3c
To be a part of challenging team, which works for the growth of an organization, explores my
potential and provides me an opportunity to enhance my knowledge and to be an asset of the
company.
Willing to relocate: Anywhere
WORK EXPERIENCE
Systems Engineer
Infosys Limited -  Chennai, Tamil Nadu -
December 2016 to Present
Trainee
Infosys Limited -  Mysore, Karnataka -
June 2016 to November 2016
Client: Renault
Project Details:
• Trained in .Net technology.
• Handled VB part of the application which has mainframe as backend and Visual basic as the
front end with annuaire service as the medium between mainframe and VB.
• Took care of site creation in the Share Point technology.
• Experienced in using SmartSVN, TortoiseSVN, DB Visualiser.
EDUCATION
B E in Production Engineering
Velammal Engineering College -  Chennai, Tamil Nadu
2016
Tamil Nadu Board

Next, we let __nlp__ process that single resumee.<br>
__Task 2__: print the results in __doc__. For each result, print the underlying text and the label.

In [14]:
doc = nlp(restxt)

# Print the results in doc. For each result, print the text and the label.
print([ (token.text, token.label)  for token in doc.ents])

[('Karthihayini C', 383), ('Rajapalaiyam', 384), ('Tamil Nadu - Email', 380), ('Infosys Limited -  Chennai', 383), ('Tamil Nadu -', 380), ('December 2016', 391), ('Trainee', 380), ('Mysore', 384), ('Karnataka', 384), ('June 2016 to November 2016', 391), ('Renault', 383), ('Handled', 386), ('VB', 386), ('Share Point', 383), ('DB Visualiser', 380), ('Production Engineering\n\n', 383), ('Tamil Nadu', 380), ('2016', 391), ('Tamil Nadu Board Of Education', 383), ('Rajapalaiyam', 380), ('Tamil Nadu', 380), ('2012', 391), ('2010', 391), ('SKILLS', 383), ('Destructive Testing', 380), ('Less than 1 year', 391), ('Less than 1 year', 391), ('Non-Destructive', 383), ('Non-Destructive Testing', 383), ('Less than 1 year', 391), ('Less than 1 year', 391), ('1', 397), ('2', 397), ('Madurantakam', 380), ('ASHOK', 383), ('CARBORUNDUM', 383), ('Thiruvottiyur', 381), ('Thirumudivakkam', 380), ('IN-PLANT TRAININGS & WORKSHOPS', 383), ('Minda Corporation Limited', 383), ('Chennai', 389), ('BHEL', 388), ('Ra

__Question 2__: How well did Spacy perform at recognizing the labels for this text?<br>

When Spacy predicted the labels for this resumee, a pretrained model was used.
<br>
__Task 3__: print for this resumee the original labels and their corresponding text.

In [15]:
# TODO print for that resumee the original labels and their corresponding text.
labeled_ents = converted_complete_resumees[resume][1]["entities"]
for ent in labeled_ents: 
  print(ent[-1],":\t", restxt[ent[0]:ent[1]] )

Skills :	 Consistent
Skills :	 Hard working
Skills :	  Flexible
Skills :	 Ability to quick grasp the concepts
Skills :	 Destructive Testing (Less than 1 year), FORGE (Less than 1 year), Non-Destructive (Less than 1
year), Non-Destructive Testing (Less than 1 year), quality control (Less than 1 year)

College Name :	 Hr. Sec. School

Location :	 Rajapalaiyam
College Name :	 Tamil Nadu Board Of Education
College Name :	 Velammal Engineering College
Degree :	 B E in Production Engineering
Companies worked at :	 Infosys Limited
Companies worked at :	 Infosys Limited
Designation :	 Systems Engineer
Email Address :	 indeed.com/r/Karthihayini-
C/627254c443836b3c
Location :	 Rajapalaiyam
Companies worked at :	 Infosys Limited

Companies worked at :	 Infosys Limited
Designation :	 Systems Engineer
Name :	 Karthihayini C


__Question 3__: Compare the performance of the pretrained model __nlp__ and the true labels. Did Spacy perform well? If not, try to explain why.<br>

It did not perform well 

__Task 4__: Remember last homework? You chose three labels. Select all the resumees, in which all three labels appear.

In [20]:
# fill in your chosen labels in chosen_entity_labels
chosen_entity_labels = [ "Skills", "College Name", "Email Address" ]

## this method gathers all resumes which have all of the chosen entites above.
def gather_candidates(dataset,entity_labels):
    candidates = list()
    for resume in dataset:
        res_ent_labels = list(zip(*resume[1]["entities"]))[2]
        if set(entity_labels).issubset(res_ent_labels):
            candidates.append(resume)
    return candidates

training_data = gather_candidates( converted_complete_resumees, chosen_entity_labels )
print("Gathered c training examples".format(len(training_data)))

Gathered c training examples


__Task 5__: Next, we want to remove all other entities, since we only want to train NER for the three entities in __chosen_entity_labels__.

In [21]:
## filter all annotation based on filter list
def filter_ents(ents, filter):
    filtered = [ent for ent in ents if ent[2] in filter]
    return filtered

## use method above to remove all but relevant (chosen) entity annotations and store in X variable. X shall contain all
## the resumees from training_data, but their entity annotations shall be filtered using the function from above.
X = [(sample[0], {"entities": filter_ents(sample[1]["entities"], chosen_entity_labels)}) for sample in training_data]


__Task 6__: Some resumees cause trouble. We filter these out with the following lines of code.<br>
First, use __add_label__ to add your chosen labels to the __ner__ model.

In [22]:
from spacy.gold import GoldParse 


print("Selected Labels: ", chosen_entity_labels )
# add your labels 

ner.add_label("Skills")

ner.add_label("College Name")

ner.add_label("Email Address")

nlp.begin_training()

good = []

for item in X:
  
  text = nlp.make_doc( item[ 0 ] )

  try:
    
    gold = GoldParse( text, entities = item[ 1 ][ "entities" ] )

  except ValueError:

    continue
  
  try:
    
    nlp.update( [ text ], [ gold ], drop = 0.3 )

  except Exception:

    pass

  else:

    good.append( item )

print( "Number of good samples: " + str( len( good ) ) )

print( "" )

print( "" )

print( "Number of bad sampples: " + str( len( X ) - len( good ) ) )

Selected Labels:  ['Skills', 'College Name', 'Email Address']
Number of good samples: 339


Number of bad sampples: 36


For a machine learning model, it is essential to be able to generalize. Only a model, that can generalize well is able to process new data in a meaningful way. Therefore, one usually separates the data set into two sets: the training set and the test set. The training set is used to train the model. The test set is used to evaluate the performance of the model on unseen data.

In [23]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split( [ item[ 0 ] for item in good ], [ item[ 1 ] for item in good ], test_size = 0.3 )

__Task 7__: Complete the following code. Shuffle __new_index__. Create the data sets __x_shuffled__ and __y_shuffled__. Use these to create minibatches, iterate over these minibatches, preprocess the data in a given minibatch using __nlp.make_doc__ and __GoldParse__. Employ __nlp.update__ to update the model using these preprocessed data.

In [24]:
import numpy as np
from spacy.util import minibatch, compounding

nlp.begin_training()

new_index = np.arange( len( x_train ) )

x_data = np.array( x_train )

y_data = np.array( y_train )

for i in range( 20 ):

  # TODO shuffle new_index
  np.random.shuffle(new_index)

  x_shuffled = x_data[new_index]  #TODO create x_shuffled from x_data by using the shuffled new_index

  y_shuffled = y_data[new_index]  #TODO create y_shuffled from y_data by using the shuffled new_index



  # divide the data in x_shuffled and y_shuffled into minibatches of identical size
  n_batch = 3
  batch_size = int(len(x_shuffled) / n_batch)
  minibatches =[]
  # iterate over these minibatches
  for b in range(n_batch):
    x_batch = x_shuffled[b*batch_size: (b+1) * batch_size]
    y_batch = y_shuffled[b*batch_size: (b+1) * batch_size]

    # preprocess the data in a minibatch using nlp.make_doc and GoldParse
    #docs = nlp.make_doc(str(x_batch))
    docs =[None]*len(x_batch)
    golds=[None]*len(y_batch)
    for i, (text_sample, label) in enumerate(zip(x_batch, y_batch)):
      docs[i] = nlp.make_doc(str(text_sample))
      golds[i]=  GoldParse(docs[i], entities=label["entities"] )
  # use these preprocessed data and nlp.update to train the model
  nlp.update( docs, golds, drop = 0.3 )

__Question 4__: Why did we shuffle the data?<br>

We shuffle the data in order to improve results and avoid to memorize the order. 

Why did we employ mini batches?<br>
For better computation efficiency and optimization

Reasearch the term __epoch__ in machine learning. How many epochs of training do we employ?<br>
__Task 8__: Next, we choose one resumee and print it.

In [29]:
n_resume = 3
resume = str(x_test[n_resume])

print( resume )

Ananya Chavan
lecturer - oracle tutorials

Mumbai, Maharashtra - Email me on Indeed: indeed.com/r/Ananya-
Chavan/738779ab71971a96

Seeking a responsible job with an opportunity for professional challenges and utilize my skills
up to its extreme.

WORK EXPERIENCE

lecturer

Oracle tutorials -  Mumbai, Maharashtra -

April 2016 to Present

for computer science (STD 11th and 12th) (2 years)
➢ Worked at "Dr.Babasaheb Ambedkar College, Chembur (W) " as a lecturer for • B.Sc. (Computer
Science & Information Technology)
• F.Y.J.C. (Computer Science & I.T.)
• S.Y.J.C. (Computer Science & I.T.)
➢ Worked at "LIVE" as a Head of the IT Department and Lecturer for Web designing.
➢ Worked at "Kohinoor College Of Hotel Management" as visiting lecturer for SEM I.
➢ Working at "ORACLE TUTORIALS" as a lecturer for computer science (STD 11th and 12th)

EDUCATION

MCA

Mumbai University -  Mumbai, Maharashtra

B.Sc. in Com.Sci

Mumbai University -  Mumbai, Maharashtra

SKILLS

SEARCH ENGINE MARKETING (2 y

__Task 9__: we process this resumee using __nlp__. Print for all items in __doc.ents__ the predicted label and the corresponding text. Then print the correct labels and their corresponding text for that resumee with data from __y_test__.

In [30]:
doc = nlp( resume )

# print for all the items in doc.ents the predicted label and the corresponding text
print(" Predicted  Labels")
for item in doc.ents:
  for entry in item:
    print(entry)

print(" Correct Labels")
# print the correct labels and their corresponding text for that resumee with data from y_test
correct_ents = y_test[n_resume]['entities']
for ent in correct_ents:
    print("{}: {}".format(ent[2], resume[ent[0]: ent[1]]))

 Predicted  Labels
 Correct Labels
Skills: EARCH ENGINE MARKETING (2 years), SEM (2 years), ACCESS (Less than 1 year), AJAX (Less
than 1 year), APACHE (Less than 1 year)

ADDITIONAL INFORMATION

Technical skills:
Languages: C, C++, Java (J2EE),
Web Component APIS:: Jdbc, Servlet, JSP.
Frameworks: Spring 4 & Struts 2
ORM Framework: Hibernate
Web Development: Html5, CSS3, Java Script, Ajax &JQuery, Angular Js

https://www.indeed.com/r/Ananya-Chavan/738779ab71971a96?isid=rex-download&ikw=download-top&co=IN
https://www.indeed.com/r/Ananya-Chavan/738779ab71971a96?isid=rex-download&ikw=download-top&co=IN


Application Servers: Apache Tomcat,
IDE: Eclipse, Netbeans
Database: Ms-Access, Mysql
Operating Systems: Windows 7, 8, 10
FTP Client: Filezilla
Versioning Tools: Git
College Name: Mumbai University
College Name: Mumbai University
Email Address: indeed.com/r/Ananya-
Chavan/738779ab71971a96



__Question 5__: What labels did the model predict correctly?<br> 
Where appeared problems?<br> 
How can you explain the problems?<br>
__Question 6__: We can evaluate the performance of the model using 4 metrics: the __Accuracy__, the __Precision__, the __Recall__ and __F1__.<br>
Inform yourself on these metrics. How are they defined? Explain the concept of __True Positive__, __True Negative__, __False Positive__ and __False Negative__. Use these to define  the __Accuracy__, the __Precision__, the __Recall__ and __F1__, and also give the formula for each of these.<br>
__Task 10__: Complete the following code. Call __make_bilou_df__ with a resume from the test set and store result in __bilou_df__ variable.

In [31]:
from spacy.gold import biluo_tags_from_offsets
import pandas as pd
from IPython.display import display, HTML

## returns a pandas dataframe with tokens, prediction, and true (Gold Standard) annotations of tokens
def make_bilou_df(nlp,resume):
    """
    param nlp - a trained spacy model
    param resume - a resume from our train or test set
    """
    doc = nlp(resume[0])
    bilou_ents_predicted = biluo_tags_from_offsets(doc, [(ent.start_char,ent.end_char,ent.label_)for ent in doc.ents])
    bilou_ents_true = biluo_tags_from_offsets(doc, [(ent[0], ent[1], ent[2]) for ent in resume[1]["entities"]])

    
    doc_tokens = [tok.text for tok in doc]
    bilou_df = pd.DataFrame()
    bilou_df["Tokens"] =doc_tokens
    bilou_df["Tokens"] = bilou_df["Tokens"].str.replace("\\s+","") 
    bilou_df["Predicted"] = bilou_ents_predicted
    bilou_df["True"] = bilou_ents_true
    return bilou_df

## call method above with a resume from test set and store result in bilou_df variable.
resume = [ str(x_test[n_resume]), y_test[n_resume] ]
bilou_df = make_bilou_df( nlp, resume )
display(bilou_df)  

,Tokens,Predicted,True
0,Ananya,O,O
1,Chavan,O,O
2,,O,O
3,lecturer,O,O
4,-,O,O
...,...,...,...
635,,O,O
636,information,O,O
637,of,O,O
638,Snakes,O,O


Inform yourself on the [BILUO](https://spacy.io/usage/linguistic-features#accessing-ner) scheme.<br>
__Question 7__: Why do you think is it better to tag entities using this scheme (consider names of humans, descriptions of items in a shop)?<br>
__Task 11__: employ pandas dataframe api to get a subset where predicted and true labels are the same. Compute the accuracy using the formula you researched above.

In [32]:
##  bilou_df is a pandas dataframe. Use pandas dataframe api to get a subset where predicted and true are the same. 
same_df = bilou_df[bilou_df["Predicted"] == bilou_df["True"]]
##  compute the accuracy (n_correct/n_total)
accuracy = len(same_df) / len(bilou_df)

print("Accuracy on one resume: ",accuracy)

Accuracy on one resume:  0.7625


The __accuracy__ is not 100%. Therefore, we want to have a look at those tokens, where the predicted and the true value differ.<br>
__Task 12__: create a dataframe diff_df where the predicted values and the true values differ

In [33]:
# TODO create a dataframe diff_df where the predicted values and the true values differ
diff_df = bilou_df[bilou_df["Predicted"] != bilou_df["True"]]
display(diff_df)

,Tokens,Predicted,True
17,indeed.com/r/Ananya-,O,-
18,,O,-
19,Chavan/738779ab71971a96,O,-
20,,O,-
185,Mumbai,O,B-College Name
...,...,...,...
349,,O,-
350,Versioning,O,-
351,Tools,O,-
352,:,O,-


Since we only considered one resumee, we now make this comparison for the whole test set.<br>
__Task 13__: Complete the following code for the computation of the overall accuracy.

In [34]:
doc_accuracy = []

for i in range( len( x_test ) ):

  resume = [x_test[i], y_test[i]]

  bilou_df = make_bilou_df(nlp,resume)

  same_df = bilou_df[bilou_df["Predicted"] == bilou_df["True"]]

  doc_accuracy.append(  len(same_df) / len(bilou_df) )

total_acc = np.mean( doc_accuracy )
print("Accuracy: ",total_acc)

Accuracy:  0.8862631649960832


So we got an __accuracy__ of about 90% on average. This is quite good considering, that we only considered about 300 cases for training.<br>
__Task 14__: Next, we want to find out, what the model did, when it went wrong. We only consider 5 resumees.<br>
Complete the following code.

In [35]:
for i in range( 5 ):

  resume = [x_test[i], y_test[i]]

  bilou_df = make_bilou_df(nlp,resume)

  difference_df =  bilou_df[bilou_df["Predicted"] != bilou_df["True"]]

  # print, where the labels from Spacy and the annotation differ. Print the text, the predicted and the true labels.

  print("Text: ", resume[0])
  print(" ----------Predicted LAbel ------ ")
  print(difference_df["Predicted"] )
  print(" ----------True Label ------ ")
  print(difference_df["True"] )


Text:  Sudaya Puranik
Principal Engineer Technical Staff - Company 1

Bengaluru, Karnataka - Email me on Indeed: indeed.com/r/Sudaya-Puranik/eaf5f7c1a67c6c38

To secure a promising position that offers both a challenge and a good opportunity for both
professional and personal growth.

Willing to relocate to: Bengaluru, Karnataka

WORK EXPERIENCE

Principal Engineer Technical Staff

Company 1 -  Bengaluru, Karnataka -

September 2005 to Present

Total Experience: 12 years 6 months.
Worked onshore and offshore for the projects. Extensive client facing and team management
skills.

Company 1: Oracle India Private Limited, Bangalore
Designation: Principal Engineer Technical Staff
Duration: September 2005 - till date

KEY TECHNICAL SKILLS
Databases: Oracle 9i/10g, MySql
Languages: PL/SQL, Shell Script.
Frameworks and Portals: Oracle ADF
Server side Technology: J2ee,
IDE: Jdeveloper 10.x

TECHNICAL EXPERIENCE SUMMARY
➢ 1 year Oracle Applications checklist implementation
➢ 3+ years of Disaster

__Question 8__: What was predicted, when the prediction differed from the true label?<br>
What do you think is necessary for computing the accuracy on token level?<br> 
What is the advantage of computing the accuracy on token level?<br>
__Task 15__: Complete the following code.

In [36]:
##  cycle through chosen_entity_labels and calculate metrics for each entity using test data
data = []
for label in chosen_entity_labels:
    ## variables to store results for all resumes for one entity type
    true_positives = 0
    false_positives = 0
    false_negatives = 0
    for i in range( len( x_test ) ):
        ## use make_bilou_df on each resume in our test set, and calculate for each entity true and false positives,
        ## and false negatives. 

        resume = [x_test[i], y_test[i]]
        
        tres_df = make_bilou_df(nlp,resume)
        tp, fp, fn = 0,0,0
        
        ## calculate true false positives and false negatives for each resume
        for k, items in tres_df.iterrows():
          if items['True'] == items['Predicted'] and  label in items['True']: 
            tp += 1 
          elif items['True'] != items['Predicted'] and  label in items['Predicted']: 
            fp += 1
          elif items['True'] != items['Predicted'] and  label in items['True']:
            fn += 1

        ## aggregate result for each resume to totals
        true_positives = true_positives + tp
        false_positives = false_positives + fp
        false_negatives = false_negatives + fn
    
    print("For label '{}' tp: {} fp: {} fn: {}".format(label,true_positives,false_positives,false_negatives))
    
    ## Use the formulas you learned to calculate metrics and print them out
    ## also: prevent division by zero without raising errors. Explain your choice
    eps = 10e-8
    precision = true_positives / (true_positives + false_positives + eps)
    recall = true_positives / (true_positives + false_negatives + eps)
    f1 = 2 * (precision * recall)/(precision + recall + eps)
    
    row = [label,precision,recall,f1]
    data.append(row)

## make pandas dataframe with metrics data. Use the chosen entity labels as an index, and the metric names as columns. 
metric_df = pd.DataFrame( data, columns = [ "Label", "Precision", "Recall", "F1" ] )
display(metric_df)

For label 'Skills' tp: 0 fp: 0 fn: 2972
For label 'College Name' tp: 0 fp: 0 fn: 529
For label 'Email Address' tp: 0 fp: 0 fn: 120


,Label,Precision,Recall,F1
0,Skills,0.0,0.0,0.0
1,College Name,0.0,0.0,0.0
2,Email Address,0.0,0.0,0.0


__Question 9__: Explain from these statistics how well __nlp__ performs.<br>
__Task 16__: Compute for each metric (Precision, Recall, F1) the mean values.

In [37]:
for label in [ "Precision", "Recall", "F1" ]:
    # Compute mean and print
    print(label, "\t: ", metric_df[label].mean())

Precision 	:  0.0
Recall 	:  0.0
F1 	:  0.0


__Question 10__: What do you learn, when you compare the performance of the model on the token level with the performance of the model on the global level from above?<br>
Next, we prepare data for flair.

In [38]:
train = [ [ x_train[ i ], y_train[ i ] ] for i in range( len( x_train ) ) ]

test = [ [ x_test[ i ], y_test[ i ] ] for i in range( len( x_test ) ) ]

__Task 17__: Complete the following code.

In [39]:

# prepare data
training_data_as_bilou = [make_bilou_df(nlp,res) for res in train]

test_data_as_bilou = [make_bilou_df(nlp,res) for res in test]


# set up paths
path_to_training_file = os.getcwd() + "/training_data.csv"

path_to_test_file = os.getcwd() + "/test_data.csv"



# make sure, that if the corresponding files exist, they are emptied
if os.path.isfile( path_to_training_file ):

  open( path_to_training_file, "w" ).close()

if os.path.isfile( path_to_test_file ):

  open( path_to_test_file, "w" ).close()


# open empty files
training_file = open( path_to_training_file, "a", encoding = "utf-8" )
    
test_file = open( path_to_test_file, "a", encoding = "utf-8" )


for item in training_data_as_bilou:
  for i, row in item.iterrows():
      token = row["Tokens"]
      #  remove all tokens like "", " ", "\n" by ignoring them
      if token in ["", " ", "\n"]:
          continue

      # for all other tokens do the following:
      label =row["True"]
      # create a string s: s = token + " " + label + "\n"
      s = token + " " 
 
      # if the label is "-", then write s = token + " O\n"
      s = s + (label if label != "-" else "0") + "\n"
      # write this newly created string to file
      training_file.write(s)
      # if this newly created string contains ".", then also write a
      # newline to file that only contains "\n"
      if "." in s:
        training_file.write("\n")

  # Using this scheme, each line in the resulting files corresponds either to an empty line or a token.
  # Flair assembles a block of nonempty lines into a sentence. Therefore, the empty line
  # is a signal for Flair that the current sentence is finished. Therefore, we extracted
  # the whitespaces above.

for item in test_data_as_bilou:
  #  the same as above.
  for i, row in item.iterrows():
    label = row["True"]
    token = row["Tokens"]
    if token in ["", " ", "\n"]:
        continue
    s = token + " " + (label if label != "-" else "0") + "\n"
    test_file.write(s)
    if "." in s:
      test_file.write("\n")

training_file.close()

test_file.close()


Start Flair

In [40]:
pip install flair

     |████████████████████████████████| 286kB 32.3MB/s 
     |████████████████████████████████| 1.2MB 37.3MB/s 
     |████████████████████████████████| 19.7MB 153kB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 776.8MB 22kB/s 
     |████████████████████████████████| 71kB 10.9MB/s 
     |████████████████████████████████| 798kB 58.6MB/s 
     |████████████████████████████████| 2.3MB 39.1MB/s 
     |████████████████████████████████| 983kB 53.2MB/s 
     |████████████████████████████████| 3.3MB 29.7MB/s 
     |████████████████████████████████| 901kB 46.2MB/s 
  Created wheel for gdown: filename=gdown-3.12.2-cp37-none-any.whl size=9693 sha256=fdb44c70584bb9183c0945ab5036ade7b0ab8352264ba52140bb2a263e1848c7
  Stored in directory: /root/.cache/pip/wheels/81/d0/d7/d9983facc6f2775411803e0e2d30ebf98efbf2fc6e57701e09
Successfully built gdown
  Created wheel for segtok: filena

In [41]:
from flair.data import Corpus
from flair.data_fetcher import NLPTaskDataFetcher


# your training file name
data_folder = os.getcwd() 

train_file = "training_data.csv"

# your training file name
test_file = "test_data.csv"

# when we wrote the data files, each row was either empty to signal the end
# of a sentence to Flair, or the line contained a token, a white space and a label.
# In the next line, we assign, that the token is the "text", and that the label is 
# "ner" label
columns =  {0: 'text', 1: 'ner'}

## Now load our csv into flair corpus
corpus = NLPTaskDataFetcher.load_column_corpus(data_folder,column_format=columns,
                                               train_file=train_file,
                                               test_file=test_file)
print(corpus)

2021-06-02 07:35:32,289 Reading data from /content/gdrive/My Drive/flair
2021-06-02 07:35:32,290 Train: /content/gdrive/My Drive/flair/training_data.csv
2021-06-02 07:35:32,296 Dev: None
2021-06-02 07:35:32,297 Test: /content/gdrive/My Drive/flair/test_data.csv


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: Call to deprecated function (or staticmethod) load_column_corpus. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.


Corpus: 6150 train + 683 dev + 3120 test sentences
